In [8]:
m = 4
n = 3

3

In [9]:
M = [i for i in 1:m]
F = [j for j in 1:n]

3-element Array{Int64,1}:
 1
 2
 3

In [10]:
using Matching
function mat2vecs{T<:Integer}(prefs::Matrix{T})
    return [prefs[1:findfirst(prefs[:, j], 0)-1, j] for j in 1:size(prefs, 2)]
end

mat2vecs (generic function with 1 method)

In [11]:
m_prefs, f_prefs = mat2vecs.(random_prefs(m, n, allow_unmatched=false))
m_prefs, f_prefs = mat2vecs.(random_prefs(m, n))

(Array

(Array{Int64,1}[[1,2],[2,1,3],[2,3],[1,2,3]],Array{Int64,1}[[1,2],[1,2,3],[2]])

In [12]:
for o in 1:m
    push!(m_prefs[o],0)
end

for k in 1:n
    push!(f_prefs[k],0)
end

{#T<:

In [13]:
m_prefs

Integer, 2}

4-element Array{Array{Int64,1},1}:
 [1,2,0]  
 [2,1,3,0]
 [2,3,0]  
 [1,2,3,0]

) in module Main

In [14]:
f_prefs

 at In[3]:3 overwritten

3-element Array{Array{Int64,1},1}:
 [1,2,0]  
 [1,2,3,0]
 [2,0]    

 at In[10]:3.


In [17]:
rank[f_prefs[1]][2]

LoadError: MethodError: no method matching getindex(::Base.LinAlg.#rank, ::Array{Int64,1})

search: rowvals



```
rowvals(A::SparseMatrixCSC)
```

Return a vector of the row indices of `A`. Any modifications to the returned vector will mutate `A` as well. Providing access to how the row indices are stored internally can be useful in conjunction with iterating over structural nonzero values. See also [`nonzeros`](:func:`nonzeros`) and [`nzrange`](:func:`nzrange`).


In [ ]:
function matching(m_prefs,f_prefs)
    
    for q in 1:end
    
    if m_prefs[][q] = 0
        
        break
        
    end
    
    if m_prefs[][q] != 0
        
        continue 
        
    end
        
        for i in 1:m
        
        if 
        
        